# Structured Flowsheet Example
The Structured Flowsheet subpackage, in idaes.core.util.structfs, provides
a way to consistently structure flowsheets and then functions to help
manipulate them interactively or from an API.

Author: Dan Gunter, LBNL

## Imports

In [11]:
# general-purpose imports
from pprint import pprint

# Import the 'FS' structured flowsheet wrapper
from flash_flowsheet import FS

# Imports for the actions
from idaes.core.util.structfs.runner_actions import Timer, UnitDofChecker

# log-related
from idaes.core.util.structfs.logutil import quiet, unquiet

## Flowsheet setup

### Add actions
Add 'actions', which are the framework that automatically does things
as the flowsheet runs. In this case, add actions for collecting timings
and calculating degrees of freedom.

In [ ]:
# Add some actions
FS.add_action("timer", Timer)
FS.add_action(
    "dof", UnitDofChecker, "fs", ["build", "solve_initial", "solve_optimization"]
)

## Solve the square problem

### Run the solver

In [15]:
# solve the square problem (up to 'solve_initial')
print("\n".join(FS.list_steps()))
print("=" * 40)
quiet()
FS.run_steps(last="solve_initial")
unquiet()

build
set_operating_conditions
initialize
set_solver
solve_initial
solve_optimization
2025-11-21 04:59:31 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 1 optimal - Optimal Solution Found.
2025-11-21 04:59:31 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 2 optimal - Optimal Solution Found.
2025-11-21 04:59:31 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 3 optimal - Optimal Solution Found.
2025-11-21 04:59:31 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 4 optimal - Optimal Solution Found.
2025-11-21 04:59:31 [INFO] idaes.init.fs.flash.control_volume.properties_in: Initialization Step 5 optimal - Optimal Solution Found.
2025-11-21 04:59:31 [INFO] idaes.init.fs.flash.control_volume.properties_out: Initialization Step 1 optimal - Optimal Solution Found.
2025-11-21 04:59:31 [INFO] idaes.init.fs.flash.control_volume.properties_out: Initialization Step 2 optimal - Optim

### Show results

In [6]:
print(FS.results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 41
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.005449056625366211
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



### Show timings

In [7]:
FS.get_action("timer")

Time per step:

build                    :    0.029  15.2%
set_operating_conditions :    0.000   0.2%
initialize               :    0.146  76.0%
set_solver               :    0.000   0.0%
solve_initial            :    0.016   8.6%

Total time: 0.209 s



### Show degrees of freedom

In [8]:
dof = FS.get_action("dof")
dof.summary()

Degrees of freedom: 0

Degrees of freedom after steps:
  build:
    fs                      : 7
    fs.flash                : 2
    fs.flash.control_volume : 7
    fs.flash.split          : 0
    fs.properties           : 0
    fs.properties.Liq       : 0
    fs.properties.Vap       : 0
    fs.properties.benzene   : 0
    fs.properties.toluene   : 0
  solve_initial:
    fs                      : 0
    fs.flash                : 0
    fs.flash.control_volume : 0
    fs.flash.split          : 0
    fs.properties           : 0
    fs.properties.Liq       : 0
    fs.properties.Vap       : 0
    fs.properties.benzene   : 0
    fs.properties.toluene   : 0


## Solve optimization

In [ ]:
# unfix the inlet temperature
temp = FS.model.fs.flash.inlet.temperature[0]
print(f"Before: {temp.value}")
temp.free()
# solve again
FS.run_steps(first="solve_optimization")
# look at new value
print(f"After : {temp.value}")

Before: 368
After : 368.85306111169916


### Show new degrees of freedom

In [17]:
dof.summary(step="solve_optimization")

Degrees of freedom: 0

fs                      : 1
fs.flash                : 0
fs.flash.control_volume : 5
fs.flash.split          : 0
fs.properties           : 0
fs.properties.Liq       : 0
fs.properties.Vap       : 0
fs.properties.benzene   : 0
fs.properties.toluene   : 0


## Done!